In [46]:
import string
import pickle
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

In [47]:
file = open("E:/Internshp/dataset.txt", "r", encoding = "utf8")
lines = []

for i in file:
    lines.append(i)
    
data = ""

for i in lines:
    data = ' '. join(lines)
    
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '')

translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
new_data = data.translate(translator)

z = []

for i in data.split():
    if i not in z:
        z.append(i)
        
data = ' '.join(z)


In [48]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

# saving the tokenizer for predict function.
pickle.dump(tokenizer, open('tokenizer1.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]

vocab_size = len(tokenizer.word_index) + 1

sequences = []

for i in range(1, len(sequence_data)):
    words = sequence_data[i-1:i+1]
    sequences.append(words)
    
sequences = np.array(sequences)

X = []
y = []

for i in sequences:
    X.append(i[0])
    y.append(i[1])
    
X = np.array(X)
y = np.array(y)

y = to_categorical(y, num_classes=vocab_size)

In [49]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense


In [50]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

In [51]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 1, 10)             30540     
                                                                 
 lstm_3 (LSTM)               (None, 1, 1000)           4044000   
                                                                 
 lstm_4 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense_3 (Dense)             (None, 1000)              1001000   
                                                                 
 dense_4 (Dense)             (None, 3054)              3057054   
                                                                 
Total params: 16,136,594
Trainable params: 16,136,594
Non-trainable params: 0
_________________________________________________________________


In [52]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard

checkpoint = ModelCheckpoint("nextword1.h5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto')

reduce = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose = 1)

logdir='logsnextword1'
tensorboard_Visualization = TensorBoard(log_dir=logdir)

In [53]:
from keras.optimizers import Adam


In [54]:
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.001))
model.fit(X, y, epochs=150, batch_size=64, callbacks=[checkpoint, reduce, tensorboard_Visualization])

Epoch 1/150
73/74 [============================>.] - ETA: 0s - loss: 8.0296
Epoch 1: loss improved from inf to 8.02966, saving model to nextword1.h5
74/74 [==============================] - 10s 76ms/step - loss: 8.0297 - lr: 0.0010
Epoch 2/150
74/74 [==============================] - ETA: 0s - loss: 7.9996
Epoch 2: loss improved from 8.02966 to 7.99956, saving model to nextword1.h5
74/74 [==============================] - 6s 76ms/step - loss: 7.9996 - lr: 0.0010
Epoch 3/150
74/74 [==============================] - ETA: 0s - loss: 7.8946
Epoch 3: loss improved from 7.99956 to 7.89459, saving model to nextword1.h5
74/74 [==============================] - 6s 76ms/step - loss: 7.8946 - lr: 0.0010
Epoch 4/150
74/74 [==============================] - ETA: 0s - loss: 7.6697
Epoch 4: loss improved from 7.89459 to 7.66974, saving model to nextword1.h5
74/74 [==============================] - 6s 76ms/step - loss: 7.6697 - lr: 0.0010
Epoch 5/150
74/74 [==============================] - ETA: 0s - 

In [59]:
# Load the trained model
model = load_model("C:/Users/Mohammed Qadir/Untitled Folder/nextword1.h5")

In [64]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 1, 10)             30540     
                                                                 
 lstm_3 (LSTM)               (None, 1, 1000)           4044000   
                                                                 
 lstm_4 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense_3 (Dense)             (None, 1000)              1001000   
                                                                 
 dense_4 (Dense)             (None, 3054)              3057054   
                                                                 
Total params: 16,136,594
Trainable params: 16,136,594
Non-trainable params: 0
_________________________________________________________________


In [78]:
from transformers import GPT2Tokenizer

# load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# read the test file
with open("E:/Internshp/test.txt", 'r', encoding='utf-8') as f:
    text = f.read()

# preprocess the text
text = text.replace('\n', ' ').replace('\r', '').strip()

# tokenize the text
encoded_text = tokenizer.encode(text, add_special_tokens=True)


In [79]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# set the device to GPU if available, otherwise CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# load the trained model
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.to(device)

# set the model to evaluation mode
model.eval()

# convert the encoded text to a PyTorch tensor
input_ids = torch.tensor(encoded_text).unsqueeze(0).to(device)

# generate text
generated_text = model.generate(input_ids, do_sample=True, max_length=3, top_k=50, top_p=0.95)
decoded_text = tokenizer.decode(generated_text.squeeze().tolist())

print(decoded_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 11, but `max_length` is set to 3. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


For writers, a random sentence can help them get their message
